In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# графики
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# для лемматизации
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['такой', 'который', 'какой', 'свой', 'мой', 'наш', 'этот', 'очень'])
stop_words.extend(['тысяча', 'сотня', 'всякий', 'любой', 'источник', 'сегодня', 'сейчас'])

# импорт датасета
df = pd.read_csv('C:\\Users\\vinov\\OneDrive\\Документы\\Универ\\Диплом\\Код\\archive\\lenta-ru-news.csv', 
                 low_memory = False, dtype = str, nrows = 400)
df.head()

# конвертировать в список
data = df.text.values.tolist()

"""for sent in data:
    # удалить эл. адреса
    s_a = re.sub(r'\S*@\S*\s?', '', sent)
#data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]

    # удалить символы новой строки
    s_a = re.sub(r'\s+', r' ', s_a)
#data = [re.sub(r'\s+', r' ', sent) for sent in data]

    # удалить кавычки
    s_a = re.sub(r'\"', '', s_a)
    
    data = re.sub(sent, s_a, data)"""

pprint(data[:2])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:2])

# построение моделей би- и триграм
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100) 
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatize(texts, allowed_postags = ['NOUN', 'ADJF', 'INFN', 'PRTF', 'GRND', 'ADV']):
    texts_out = []
    for sent in texts:
        butyavka = morph.parse(str(sent))[0]
        if butyavka.tag.POS in allowed_postags:
            texts_out.append(butyavka.lexeme)
    return texts_out

def lemmatize(texts, allowed_postags = ['NOUN', 'ADJF', 'INFN', 'PRTF', 'GRND', 'ADVB']):
    texts_out = []
    for sent in texts:
        to1 = []
        for part in sent:
            butyavka = morph.parse(part)[0]
            if butyavka.tag.POS in allowed_postags:
                to1.append(butyavka.normal_form)
        texts_out.append(to1)
    return texts_out



# биграмы
data_words_bigrams = make_bigrams(data_words)

# лемматизация
data_lemmatized = lemmatize(data_words_bigrams)
print(data_lemmatized[:2])

# убрать стоп-слова
data_nonstop = remove_stopwords(data_lemmatized)

# создание словаря
id2word = corpora.Dictionary(data_nonstop)

# создание корпуса
texts = data_nonstop
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:2])

# LDA модель
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# вычисление сложности
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # чем ниже, тем лучше

# расчёт согласованности
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_nonstop, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# визуализация
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

['Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, '
 'приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В '
 'артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 '
 'сентября огонь достиг значительного напряжения. Попытка германской пехоты '
 'пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая '
 'колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот '
 'нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона '
 'Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев '
 'обнаруживается полное перемешивание их частей, захватываются новые партии '
 'пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили '
 'неприятельский отряд, взяли его артиллерию и много пленных и, продолжая '
 'преследовать, вступили в пределы Венгрии. \n'
 '«Русский инвалид», 16 сентября 1914 года.',
 'Министерство народного просвещения, в виду 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.299794 -0.191401       1        1  28.602975
11     0.303415  0.041897       2        1  18.618562
19     0.150901  0.071107       3        1   8.016072
0      0.088043  0.053683       4        1   5.605655
18     0.055901  0.163105       5        1   5.343643
4      0.052933 -0.013121       6        1   5.313009
9      0.023307  0.005800       7        1   4.752662
7      0.053533 -0.155226       8        1   4.728463
10     0.068259  0.189826       9        1   4.679189
2     -0.024133 -0.074644      10        1   2.953867
3     -0.043745  0.082760      11        1   2.651611
14    -0.081461 -0.065229      12        1   2.490500
8     -0.100234 -0.025467      13        1   2.115932
1     -0.114660 -0.027294      14        1   1.430546
12    -0.125530 -0.009199      15        1   1.025133
13    -0.126973 -0.010860      16        1   0.516616
15    -0.122692 -0.009789      17        1   0.481112
16    -0.123721 -0.009822      18        1   0.374441
6     -0.118379 -0.008436      19        1   0.196714
17    -0.114559 -0.007690      20        1   0.103298, topic_info=               Term        Freq       Total Category  logprob  loglift
1366            сша  234.000000  234.000000  Default  30.0000  30.0000
441         человек  239.000000  239.000000  Default  29.0000  29.0000
391           взрыв  142.000000  142.000000  Default  28.0000  28.0000
1019          чечня   88.000000   88.000000  Default  27.0000  27.0000
937          боевик   84.000000   84.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
3460       кристина    0.004715    2.244505  Topic20  -9.1331   0.7099
3454          бухта    0.004709    2.244557  Topic20  -9.1345   0.7084
3466            мыс    0.004706    3.967284  Topic20  -9.1350   0.1383
2673   тысяч_доллар    0.004705   18.859397  Topic20  -9.1352  -1.4208
822   администрация    0.004700   35.962691  Topic20  -9.1363  -2.0674

[993 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
384       8  0.082148    авария
384       9  0.903625    авария
511       1  0.029608    август
511       3  0.059216    август
511       5  0.177649    август
...     ...       ...       ...
1969     14  0.200188   юстиция
2825     16  0.460068    яблоко
509      11  0.872553  японский
2600     13  0.288953      ясно
2600     14  0.577905      ясно

[1483 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 12, 20, 1, 19, 5, 10, 8, 11, 3, 4, 15, 9, 2, 13, 14, 16, 17, 7, 18])